In [1]:
!pip install pandas
!pip install -q tensorflow_text

     |████████████████████████████████| 8.3MB 2.7MB/s 
     |████████████████████████████████| 421.8MB 30kB/s 
     |████████████████████████████████| 450kB 50.0MB/s 
     |████████████████████████████████| 3.9MB 37.6MB/s 
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [9]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text


pd.set_option('max_colwidth', 150)  # Increase column width
data = pd.read_excel("Chatbot_qa.xlsx", encoding='utf8')
data.head()

,Context,Answer,pets
0,What is a coronavirus?,Coronaviruses are a large family of viruses which may cause illness in animals or humans.,0
1,What is a coronavirus?,"In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases",1
2,What is COVID-19?,COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new virus and disease were unknown before the outbreak...,1
3,What are the symptoms of COVID-19?,"The most common symptoms of COVID-19 are fever, tiredness, and dry cough. Some patients may have aches and pains, nasal congestion, runny nose, so...",1
4,What are the symptoms of COVID-19?,Some people become infected but don’t develop any symptoms and don't feel unwell. Most people (about 80%) recover from the disease without needing...,1


In [0]:
# Load module containing USE
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')
# Create response embeddings
response_encodings = module.signatures['response_encoder'](
        input=tf.constant(data.Answer),
        context=tf.constant(data.Context))['outputs']

In [5]:
import numpy as np
from IPython.core.display import display, HTML

test_questions = [
    "How should I wash my hands?",
    "Can my pet snake give me the corona?",
    "How are we sure that the chinese didn't create the coronavirus in a secret lab?",
    "What should I do when I feel sick?",
    "Is it possible to social distance and still get sick?",
    "When will the coronavirus end?",
    "When will a vaccine be made?",
    "Who first discovered the spikes protein?"
]

question_num = 1

# Create encodings for test questions
question_encodings = module.signatures['question_encoder'](
    tf.constant(test_questions)
)['outputs']

qa_inner_prods = np.inner(question_encodings, response_encodings)

# Get the responses
test_responses = data.Answer[np.argmax(qa_inner_prods, axis=1)]

# Get the top N matching questions
N = 5

qa_array = qa_inner_prods[question_num] # inner product between question and all response encodings
top_N_response_indices = np.argpartition(qa_array, -N)[-N:] 
sorted_top_response_indicies = top_N_response_indices[np.argsort(qa_array[top_N_response_indices])[::-1][:N]]
top_possible_questions = pd.Series.to_numpy(data.Context[sorted_top_response_indicies])
top_possible_questions = np.insert(top_possible_questions, N, 'None of the above questions are similar to what I asked')

frame = {'Which is your question?': top_possible_questions}
questions_df = pd.DataFrame(frame, index=np.arange(1, N+2, 1))
questions_df.to_html('possible_questions.html')
display(HTML('possible_questions.html'))

matching_question = 1
print(data.Context[sorted_top_response_indicies[matching_question-1]])
print(data.Answer[sorted_top_response_indicies[matching_question-1]])



# Show them in a dataframe
pd.DataFrame({'Test Questions': test_questions, 'Test Responses': test_responses})

,Which is your question?
1,Can humans be infected with the COVID-19 from an animal source?
2,What is a coronavirus?
3,Can humans get coronavirus from dogs or cats?
4,Can coronavirus live on objects?
5,Can humans be infected with the COVID-19 from an animal source?
6,None of the above questions are similar to what I asked


Can humans be infected with the COVID-19 from an animal source?
Coronaviruses are a large family of viruses that are common in animals. Occasionally, people get infected with these viruses which may then spread to other people.


,Test Questions,Test Responses
13,How should I wash my hands?,Regularly and thoroughly clean your hands with an alcohol-based hand rub or wash them with soap and water.
45,Can my pet snake give me the corona?,"Coronaviruses are a large family of viruses that are common in animals. Occasionally, people get infected with these viruses which may then spread..."
2,How are we sure that the chinese didn't create the coronavirus in a secret lab?,COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new virus and disease were unknown before the outbreak...
17,What should I do when I feel sick?,"Stay home if you feel unwell. If you have a fever, cough and difficulty breathing, seek medical attention and call in advance. Follow the directio..."
39,Is it possible to social distance and still get sick?,If you are not ill or looking after someone who is ill then you are wasting a mask.
52,When will the coronavirus end?,"It is not certain how long the virus that causes COVID-19 survives on surfaces, but it seems to behave like other coronaviruses."
32,When will a vaccine be made?,"Not yet. To date, there is no vaccine and no specific antiviral medicine to prevent or treat COVID-2019."
49,Who first discovered the spikes protein?,"Handle raw meat, milk or animal organs with care to avoid contamination of uncooked foods and avoid consuming raw or undercooked animal products."


In [15]:
from googlesearch import search

def get_top_questions(qa_array, N = 5):
  top_N_response_indices = np.argpartition(qa_array, -N)[-N:] 
  sorted_top_response_indicies = top_N_response_indices[np.argsort(qa_array[top_N_response_indices])[::-1][:N]]
  top_possible_questions = pd.Series.to_numpy(data.Context[sorted_top_response_indicies])
  top_possible_questions = np.insert(top_possible_questions, N, 'None of the above questions are similar to what I asked')
  
  frame = {'Which is your question?': top_possible_questions}
  questions_df = pd.DataFrame(frame, index=np.arange(1, N+2, 1))
  return questions_df

def get_response(x):
  # TODO:
  # Do I/O here
  # return numbers as ints, no/yes as "no" or "yes"
  if (x == 0): return 'no'
  if (x == 1): return 6

def get_label_from_question(question):
  # TODO: get label from question
  return 'pets'

def get_question():
  # TODO:
  # get question from I/O
  # return a string
  return "Can my pet snake give me the corona?"

def is_exit(question):
  if (question == 'bye'):
    return True
  return False

def get_google_search(query):
  search_result_list = list(search(query, tld="com", num=10, stop=3, pause=1))
  return search_result_list[0]

def convo(question):
  label = get_label_from_question(question)

  filtered_data = data.loc[data[label] == 1, ['Context', 'Answer']]
  response_encodings = module.signatures['response_encoder'](
        input=tf.constant(filtered_data.Answer),
        context=tf.constant(filtered_data.Context))['outputs']

  question_encodings = module.signatures['question_encoder'](
    tf.constant(question)
  )['outputs']

  print("********")
  print(question)
  print("********")

# Determine scores for question and answer encoding products
  qa_inner_prods = np.inner(question_encodings, response_encodings)

  print(data.Answer[np.argmax(qa_inner_prods, axis=1)])
  print("Did I answer your question?")
  response = get_response(0)

  print("********")
  print(response)
  print("********")

  if (response == 'no'):
    N = 5
    questions_df = get_top_questions(qa_inner_prods[0], N)

    print("Are any of the following the questions you asked?")
    questions_df.to_html('possible_questions.html')
    display(HTML('possible_questions.html'))
    print("Please type the number corresponding to the closest question you asked")
    response = get_response(1)

    print("********")
    print(response)
    print("********")

    top_N_response_indices = np.argpartition(qa_array, -N)[-N:] 
    sorted_top_response_indicies = top_N_response_indices[np.argsort(qa_array[top_N_response_indices])[::-1][:N]]

    if (response >= 1 and response <= 5):
      print(data.Context[sorted_top_response_indicies[response-1]])
      print(data.Answer[sorted_top_response_indicies[response-1]])   
    else:
      print("Here's a link that hopefully answers your question")
      print("If this does not answer your question, please try rephrasing your question")
      print(get_google_search(question))
      return
  print("What would you like to know next about Covid-19? If you want to stop talking to me, simply type 'bye'")
  return

def main():
  print("Hi, what would you like to know about Covid-19?")
  
  count = 0

  while(True):
    if (count == 1): break
    question = get_question()
    if (is_exit(question)):
      break
    convo(question)
    count = count + 1
  print("Bye now!")

  
main()


Hi, what would you like to know about Covid-19?
********
Can my pet snake give me the corona?
********
0    Coronaviruses are a large family of viruses which may cause illness in animals or humans.
Name: Answer, dtype: object
Did I answer your question?
********
no
********
Are any of the following the questions you asked?


,Which is your question?
1,What is a coronavirus?
2,What is a coronavirus?
3,How does COVID-19 spread?
4,What are the symptoms of COVID-19?
5,What are the symptoms of COVID-19?
6,None of the above questions are similar to what I asked


Please type the number corresponding to the closest question you asked
********
6
********
Here's a link that hopefully answers your question
If this does not answer your question, please try rephrasing your question
https://www.ethosvet.com/blog-post/covid-19-with-reptiles-and-amphibians/
Bye now!
